### `Minelli Giovanni`
# Moving objects removal
The project goal is to analyze an input video and return the static background of the scene comprehensive of small dynamic changes over time. The algorithm is supposed to work properly both with input from a static camera and with a free moving camera (dynamic shot). Also, the algorithm works on-the-fly without preassumptions or preprocessing of the video frames.

In [1]:
from termcolor import colored
import numpy as np
import cv2
from matplotlib.gridspec import GridSpec
from matplotlib import pyplot as plt
from scipy import ndimage, misc
from skimage.metrics import structural_similarity as ssim
from skimage import filters
import time
import progressbar

In [2]:
def drawlines(img1,img2,lines,pts1,pts2):
    ''' Utility for showing the correspondances between detected points
        img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r,c = img1.shape
    img1 = cv2.cvtColor(img1,cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        # img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv2.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv2.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2

<img src="docs/auto_canny.png" height="400" />

In [3]:
def auto_canny(image, sigma=0.33):
	''' Apply Canny edge detector and thresholding using a lower and upper boundary on the gradient values.
	Lower value of sigma indicates a tighter threshold, whereas a larger value of sigma gives a wider threshold '''

	# compute the median of the single channel pixel intensities
	v = np.median(image)
	# apply automatic Canny edge detection using the computed median
	lower = int(max(0, (1.0 - sigma) * v))
	upper = int(min(255, (1.0 + sigma) * v))
	edged = cv2.Canny(image, lower, upper)
	# return the edged image
	return edged

- https://docs.opencv.org/4.5.1/d5/d0f/tutorial_py_gradients.html
- https://en.wikipedia.org/wiki/Image_gradient

`"The gradient at each image point is a 2D vector with the components given by the derivatives in the horizontal and vertical directions. Each gradient vector points in the direction of largest possible intensity increase."`
<img src="docs/count_oriented.png" height="800" />

An high threshold is applied on the given image to keep only the point were an high gradient is found and therefore we suspect the presence of the border of the object. The input image is an intermediary step of the `spot_the_diff` function

In [4]:
def count_oriented(img, show=False):
  ''' Count the points in a binary image taking in consideration the orientation '''

    # sobel derivatives
  derivX = cv2.Sobel(img, cv2.CV_32F, 1, 0)
  derivY = cv2.Sobel(img, cv2.CV_32F, 0, 1)

  mag = cv2.magnitude(np.absolute(derivX), np.absolute(derivY)) # absolute since a limitation of opencv from docs
  
    # thresholding of the magnitude values
  thresh = 1000
  _, mask = cv2.threshold(mag, thresh, 255, cv2.THRESH_BINARY)
  mask = np.uint8(mask>0)
  
  if show:
    plt.figure(figsize=(15,10)); plt.title("Mask magnitude min: {} max: {}".format(np.int(np.min(mag[mag>0])), np.int(np.max(mag))))
    plt.imshow(cv2.cvtColor(mask*255, cv2.COLOR_BGR2RGB))
    plt.show()
  
  return np.sum(mask)

In [5]:
def expand_borders(m): return cv2.dilate(m, None, iterations=5)
def enhance_borders(im, k=np.array([[-1,-1,-1], [-1, 9,-1], [-1,-1,-1]])): return cv2.GaussianBlur(im, (3, 3), 0) # cv2.filter2D(im, -1, k) #

**This functions is used to determine if the bound (the next_bound) around the moving object is done correctly evaluating the difference between the two images in the ipotetical static zone.**
<img src="docs/distances.png" height="1000" />
Confronting different functions of distance and configuration for thresholding or enhancing the differences i decided to use the structural similarity for a low points detection and for the aim of disambiguate over a possible equality of the images. In case of an evident presence of a difference between the two a more "precise" function is used to get the shape of the moving object.
- absdiff (double check the equality with an higher thresh value)
- Morphological transformation: close 
- Morphological transformation: gradient on the border of the shape found
- Canny of both images using the gradient as mask
- count_oriented of edges found by Canny and sum

The image with the higher value is supposed to contain "the object". The return value is unbalanced by 0.01% to prioritize the False negative and minimize the False positive.

<img src="docs/spot_the_diff.png" height="1000" />

In [31]:
index = 0

def spot_the_diff(image1_gray, image2_gray, log=False, show_work=True, debug_init=True):
    ''' Given two images return the one with the object  
          image1_gray: [prev_bound - next_bound] * prev_show
          image2_gray: [prev_bound - next_bound] * next_frame
    ret=> if bound was correct the diff function will find dirt in image1 (=>1) or image1 and image2 are equals (=>0)
          if bound was incorrect the diff function will contain some moving parts of image2 (=>2)
             (can happen that also identifies small part like light changes of image1 but returns 2) ''' 

    if debug_init:
        global index
        plt.figure(figsize=(15,15))
        plt.subplot(121); plt.title("PREV {}".format(index))
        plt.imshow(cv2.cvtColor(image1_gray, cv2.COLOR_BGR2RGB))
        plt.subplot(122); plt.title("NEXT")
        plt.imshow(cv2.cvtColor(image2_gray, cv2.COLOR_BGR2RGB))
        plt.show()
        plt.imsave("spot_debug/prev_{}.jpg".format(index),cv2.cvtColor(image1_gray, cv2.COLOR_BGR2RGB))
        plt.imsave("spot_debug/next_{}.jpg".format(index),cv2.cvtColor(image2_gray, cv2.COLOR_BGR2RGB))

    (score, sim) = ssim(image1_gray, image2_gray, full=True)
    sim = np.uint8(sim*255)
    thresh_sim =  np.uint8(np.logical_not(cv2.threshold(sim, 205, 255, cv2.THRESH_BINARY)[1])*255)
    thresh_sim = cv2.medianBlur(thresh_sim, 7)
    sum_sim = np.sum(thresh_sim>0)

      # lower bound (low thresh, high detection)
    if sum_sim<100:
        print(colored("Equality_l", 'blue'))
        return 0

    dif = cv2.absdiff(image1_gray, image2_gray)
    thresh_dif = cv2.medianBlur(dif, 5)

      # higher bound (high thresh, low detection)
    if not np.any(thresh_dif>50):
        print(colored("Equality_h", 'blue'))
        return 0

    if show_work:
        plt.figure(figsize=(20,20))
        plt.subplot(131); plt.title("ssim")
        plt.imshow(cv2.cvtColor(np.uint8(thresh_sim), cv2.COLOR_BGR2RGB))

    adaptive=0
    detected = 110
    a=None
    while(detected>80 if detected<100 else detected>105):
        a = np.uint8(thresh_dif>adaptive)*255
        detected = ((np.sum(a>0)/sum_sim)*100).round(3)
        if log: print("Not {}%".format(detected.round(2)))
        adaptive+=10

    mask = cv2.morphologyEx(a,cv2.MORPH_CLOSE, np.ones((12,12),np.uint8))
    mask = cv2.medianBlur(mask,3)
    gradient = cv2.morphologyEx(mask, cv2.MORPH_GRADIENT, np.ones((6,6),np.uint8))

    if show_work:
        plt.subplot(132); plt.title("Mask closed [{}]".format(adaptive-10))
        plt.imshow(cv2.cvtColor(np.uint8(mask), cv2.COLOR_BGR2RGB))
        plt.subplot(133); plt.title("Mask gradient [{}%]".format(detected.round(1)))
        plt.imshow(cv2.cvtColor(np.uint8(gradient), cv2.COLOR_BGR2RGB))
        plt.show()

      # canny in the border of the movement areas
    blurred1 = enhance_borders(image1_gray*(gradient>0))
    blurred2 = enhance_borders(image2_gray*(gradient>0))

    edges1 = auto_canny(blurred1)
    edges2 = auto_canny(blurred2)

      # count of canny oriented pixels in the border of the movement areas 
    i1=count_oriented(edges1)
    i2=count_oriented(edges2)
    if log: print(i1)
    if log: print(i2)

    if show_work:
        plt.figure(figsize=(20,20))
        plt.subplot(121)
        plt.title("Canny 1 [{}]".format(i1))
        plt.imshow(cv2.cvtColor(edges1, cv2.COLOR_BGR2RGB))
        plt.subplot(122)
        plt.title("Canny 2 [{}]".format(i2))
        plt.imshow(cv2.cvtColor(edges2, cv2.COLOR_BGR2RGB))
        plt.show()

    if show_work:
        plt.figure(figsize=(20,20))
        plt.subplot(121); plt.title("index {} sim_score {}".format(index, (score*100).round(1), 15), fontsize=20); 
        plt.imshow(cv2.cvtColor(image1_gray*(mask>0), cv2.COLOR_BGR2RGB))
        plt.subplot(122)
        plt.imshow(cv2.cvtColor(image2_gray*(mask>0), cv2.COLOR_BGR2RGB))

      # prioritize the false negative cases to minimize the false positives 
    if i1>(i2+(0.01*i2)):
        if show_work: plt.title("Oggetto in 1", fontsize=20); plt.show()
        print (colored("#Oggetto in 1 [{}]".format(i1-i2),'blue')); return 1;
    else:
        if show_work: plt.title("Oggetto in 2", fontsize=20); plt.show()
        print(colored("#Oggetto in 2 [{}]".format(i2-i1),'blue')); return 2;

**Detection of the camera movement (prev frame to next frame) by SIFT feature points.**

Assuming a static camera the points found in the background image will always be the same. If there is a good amount of cordinates matching between the background image and the next frame then no waping will be perfomed.
<img src="docs/sift_moving.png" height="600" />


Otherways recalculate foundamental matrix from eight-point method using only background feature points (computing the distance of point from epipolar line <a href="https://www.mdpi.com/2076-3417/10/1/268">[Jung, S.; Cho, Y.; Kim, D.; Chang, M. Moving Object Detection from Moving Camera...]</a>)
<img src="docs/point_line_distance_formula.png" />
Moreover can be applied a RANSAC algorithm to clean result from outliers. And then compute the image transformation of both the previous frame and the background image.

In [21]:
des_recovery = None

def warp(prev_show, prev_frame, frame, show_work=False, show_result=False):
      # find the keypoints and descriptors with SIFT
    global des_recovery
    global mean_prev

    if not des_recovery is None:
        kp1, des1 = des_recovery
    else:
        kp1, des1 = sift.detectAndCompute(prev_show,None)
        des_recovery = (kp1, des1)
    kp2, des2 = sift.detectAndCompute(frame,None)
    
      # determine if camera is static, 
    if len(kp1)!=0 and len(kp2)!=0:
      pp1 = np.array([[int(p.pt[0]),int(p.pt[1])] for p in kp1])
      pp2 = np.array([[int(p.pt[0]),int(p.pt[1])] for p in kp2])
      matches = int(np.sum((pp1[:,None] == pp2).all(2).any(1))/len(kp1)*100)
      if(matches<THRESH_STATIC_OR_MOVING): # the prev images need warping and old SIFT points are trashed
          print( colored("Warping", "cyan") )
          des_recovery = None
      else: # no need to warp
          return (prev_show, prev_frame, frame)

    matches1 = []; matches1_bg = []
    matches2 = []; matches2_bg = []
      # match the descriptors and build the corrspondences arrays of good matches
    matches_pairs = flann.knnMatch(des1, des2,k=2)
    for i,(m,n) in enumerate(matches_pairs):
        if m.distance < 0.7*n.distance:
            matches1.append(kp1[m.queryIdx].pt)
            matches2.append(kp2[m.trainIdx].pt)
    matches1 = np.asarray(matches1, np.int32)
    matches2 = np.asarray(matches2, np.int32)

    if len(matches1)>MIN_MATCH_COUNT:
        F, mask = cv2.findFundamentalMat(matches1,matches2,cv2.FM_LMEDS)
          # we select only inlier points
        matches1 = matches1[mask.ravel()==1]
        matches2 = matches2[mask.ravel()==1]

          # find epilines corresponding to points in left image (first image) and drawing its lines on right image
        lines2 = cv2.computeCorrespondEpilines(matches1.reshape(-1,1,2),1,F)
        lines2 = lines2.reshape(-1,3)

        if show_work:
            img3,img4 = drawlines(prev_show,frame,lines2,matches2,matches1)
            plt.figure(figsize=(20,10))
            plt.subplot(121); plt.title("lines {0} points {1}".format(len(lines2), len(matches1))); plt.imshow(img3)
            plt.subplot(122); plt.imshow(img4)
            plt.show()

          # keep only points near <1px to an epipolar line
        for pt1,pt2 in zip(matches1,matches2):
            pixels_err = PIXELS_ERR
            for line in lines2:
                pixels_err = min(pixels_err,abs(line[0] * pt1[0] + line[1] * pt1[1] + line[2]) / np.sqrt((line[0] * line[0]) + (line[1]*line[1])))
            if pixels_err<1:
                matches1_bg.append(pt1)
                matches2_bg.append(pt2)

        print("bg matches: ",len(matches1_bg),"/",len(matches1))

    if len(matches1_bg)>MIN_MATCH_COUNT:
          # retrieving perspective homography
        src = np.float32(matches1_bg)
        dst = np.float32(matches2_bg)
        H, masked = cv2.findHomography(src,dst, cv2.RANSAC, 5.0)
        dst_show = cv2.warpPerspective(np.uint16(prev_show)+1,H,(prev_frame.shape[1], prev_frame.shape[0]))
        dst = cv2.warpPerspective(np.uint16(prev_frame)+1,H,(prev_frame.shape[1], prev_frame.shape[0]))
        
          # correct the last pixel at margins
        dst_show[cv2.dilate(np.uint8(dst_show==0), None)>0]=0
        dst[cv2.dilate(np.uint8(dst==0), None)>0]=0

          # warp
        prev_show = np.uint8(((dst_show-1)*(dst_show>0))+((dst_show<1)*frame))
        prev_frame = np.uint8(((dst-1)*(dst>0))+((dst<1)*frame))
        mean_prev = [prev_show]

        if show_result:
            plt.figure(figsize=(20,10))
            plt.subplot(121); plt.title.set_text('Left warp')
            plt.imshow(cv2.cvtColor(dst,cv2.COLOR_GRAY2RGB))
            plt.subplot(122); plt.title.set_text('Compelete prev frame warped')
            plt.imshow(cv2.cvtColor(prev_frame,cv2.COLOR_GRAY2RGB))
            plt.show()
        
        return (prev_show, prev_frame, frame)
    else:
        print( colored("Not enough matches are found - {}/{}".format(len(matches1_bg), MIN_MATCH_COUNT), "red") )
        mean_prev = [prev_show]

        return None

**FLOW**

The basic understanding of the algorithms is the idea of recognize objects in movement confronting just two frames at each cycle. Found a moving object with a proper diff strategy we can confront the bound of the oject in movement with the bound found at the cycle before and then make assumptions on the movement: (↓↑←→): in a direction, (B): backwards or (F): forward.
.  
- (↓↑←→) In this case we can gain "good pixels" both from the previous frame (in the B zone) and from the next frame (in the A zone)

<img src="docs/sliding.png" />

- (B) In this case we can take pixels from the next frame in the zone of previous movement (A zone) 

<img src="docs/backward.png" />

- (C) all the movement zone remain unused

Those regions of improvement can suffer of bad bounds detection and also of the presence of old pixels of moving objects in the background image to show (due to initialization with the the first frame). To disambiguate between those situations the function `spot_the_diff` is used to confront the prev_show (the background image) and the next_frame. The response can be "equality" (under a thresh value) and so we can use the newer pixels for the update, or it can indicate the image in which the object is.

Moreover all the other static pixels (not detected as movement in the two frames difference) are used for the update of the background.


In [30]:
''' Hyperp to tune properly '''
 # FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)
sift = cv2.SIFT_create()
flann = cv2.FlannBasedMatcher(index_params,search_params)
 # WARP parameters
THRESH_STATIC_OR_MOVING = 12 # %
PIXELS_ERR = 1
MIN_MATCH_COUNT = 10
 #other parameters
sim_thresh = 165
min_rect_body_ratio_thresh = 0.005
min_rect_thresh = (5, 10)
max_bounds_per_zone = 10
frames_to_skip = 2
mean_prev_size = 5

show_work = False
debug_spot = False
show_result = False
print_log = True

 # iterators
index = 0
prev_bounds = []
frame = None
prev_frame = None
des_recovery = None
output_video = []
mean_prev = []
        

# video_name=files[np.random.randint(0,len(files))]
video_name="ex/test.avi"

cap = cv2.VideoCapture(video_name)
tot_frames = 0
while True:
    for _ in range(frames_to_skip): ret, frame_BGR = cap.read() # skip
    ret, frame_BGR = cap.read()
    if frame_BGR is None: break
    tot_frames+=1
cap.release()

cap = cv2.VideoCapture(video_name)
bar = progressbar.ProgressBar(maxval=tot_frames, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
# for i in range(100):
#     for _ in range(frames_to_skip): ret, frame_BGR = cap.read() # skip
#     ret, frame_BGR = cap.read()
while True:
    for _ in range(frames_to_skip): ret, frame_BGR = cap.read() # skip
    ret, frame_BGR = cap.read()
    if frame_BGR is None: break
    
    # restart
    start = time.time()
    index+=1
    prev_frame = frame
    frame = cv2.cvtColor(np.uint8(frame_BGR),cv2.COLOR_BGR2GRAY)
    if index<2:
        min_rect_body_ratio_thresh = (frame.shape[0] * frame.shape[1] ) * min_rect_body_ratio_thresh
        prev_show = frame.copy()
        mean_prev.append(prev_show.copy())
        continue
    handle_camera_movement = warp(prev_show, prev_frame, frame)
    if handle_camera_movement is None:
        frame = prev_frame
        continue

    prev_show, prev_frame, next_frame = handle_camera_movement
    
      # identify movement areas in prev_frame to next_frame 
    (score, sim) = ssim(prev_frame, cv2.GaussianBlur(next_frame, (3, 3), 0), full=True)
    sim = np.uint8(sim*255)
    thresh = cv2.threshold(sim, sim_thresh, 1, cv2.THRESH_BINARY)[1]
    thresh = np.uint8(np.logical_not(thresh))
    thresh = cv2.medianBlur(thresh, 11) # remove small pieces
    thresh = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE, np.ones((9,9),np.uint8)) # unify pieces

    static_area = np.uint8(np.logical_not(thresh))
    movement_area = thresh

      # get rid of the frame margins
    if np.any(movement_area[0,10:-10]) and not np.any(movement_area[10,10:-10]):
        movement_area[:10,:] = 0
    if np.any(movement_area[-1,10:-10]) and not np.any(movement_area[-10,10:-10]):
        movement_area[-10:-1,:] = 0
    if np.any(movement_area[10:-10,0]) and not np.any(movement_area[10:-10,10]):
        movement_area[:,:10] = 0
    if np.any(movement_area[10:-10,-1]) and not np.any(movement_area[10:-10,-10]):
        movement_area[:,-10:-1] = 0
    
      # crete rectangles from countours
    rects = []
    bounds = []
    recovery_bounds = []
    contours, hierarchy = cv2.findContours(movement_area,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    bounded_image=movement_area*prev_frame
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        if w > min_rect_thresh[0] and h > min_rect_thresh[1]:
            m=np.zeros_like(movement_area, np.uint8)
            m[y:y+h,x:x+w] = 1
            rects.append({"x":x,"y":y,"w":w,"h":h,"m":m})
        
    if print_log: print("rects: ", len(rects))

      # create indipendent bounds from rectangles
    while len(rects)!=0:
        x = rects[0]["x"];y = rects[0]["y"];w = rects[0]["w"];h = rects[0]["h"]
        m = rects[0]["m"]
        m_expanded = expand_borders(m) 
        cv2.rectangle(bounded_image,(x,y),(x+w,y+h),(255,255,0),2)
        rects.remove(rects[0])

        shape_modified = True
        while shape_modified:
            shape_modified = False
            for r_in in rects:
                if np.any(np.logical_and(m_expanded, r_in["m"])):
                    rects.remove(r_in)
                    cv2.rectangle(bounded_image,(r_in["x"],r_in["y"]),(r_in["x"]+r_in["w"],r_in["y"]+r_in["h"]),(115,115,0),1)
                    m[r_in["y"]:r_in["y"]+r_in["h"],r_in["x"]:r_in["x"]+r_in["w"]] = 1
                    shape_modified = True
                    break
        bounds.append(m)

    if print_log: print("bounds: ", len(bounds))

      # get rid of the distorsion due to low movement or stopping object
    for prev_bound in prev_bounds:
        movement_zones = []
        for idx_bound in range(len(bounds)):
            if np.array_equal(np.logical_and(prev_bound, bounds[idx_bound]), prev_bound):
                movement_zones.append(idx_bound)
        if len(movement_zones)>=max_bounds_per_zone: # (ignore it) push forward the previous bound to the next round
            for idx_zone in reversed(movement_zones):
                bounds.pop(idx_zone)
            recovery_bounds.append(prev_bound)
        elif len(movement_zones)>=2: # (use it) group and add the rectangle
            canvas = np.zeros(movement_area.shape)
            
            for idx_bound in reversed(movement_zones):
                canvas[bounds[idx_bound]>0] = 1
                bounds.pop(idx_bound)

            (y, x) = np.where(canvas == 1)
            (topy, topx) = (np.min(y), np.min(x))
            (bottomy, bottomx) = (np.max(y), np.max(x))
            cv2.rectangle(bounded_image,(topx,topy),(bottomx,bottomy),(115,115,0),3)
            canvas[topx:bottomx+1, topy:bottomy+1] = 1
            bounds.append(canvas)

      # improvement areas 
    a = [] # areas to ignore into zone [prev_bounds - next_bounds]. The remaining is updated with next_frame
    b = [] # areas to update into zone [next_bounds - prev_bounds]. Update with prev_frame
    recover_prev_bound = [0]*len(prev_bounds)
    if not (prev_bounds is None) or len(prev_bounds) != 0:
        for bound in bounds:
            a_current = []
            b_current = []
            recover_current_bound = []
            for prev_bound_id,prev_bound in enumerate(prev_bounds):
                if (np.sum(prev_bound[bound==1])/np.sum(bound))>0.5 or (np.sum(bound[prev_bound==1])/np.sum(prev_bound))>0.5:
                    if print_log: print("--cycle-- bound matching")
                    if np.any(np.logical_and(prev_bound, bound)):
                        if print_log: print("crossing")

                        static_prev_zone = np.uint8(np.logical_and(prev_bound, np.logical_not(bound)))
                        moving_next_zone = np.logical_and(np.logical_not(prev_bound), bound)
                        # if not np.all(prev_bound[bound==1]) and not np.all(bound[prev_bound==1]):
                        if np.any(static_prev_zone) and np.any(moving_next_zone):
                            imm = spot_the_diff(static_prev_zone*prev_show, static_prev_zone*next_frame, log=print_log, show_work=show_work, debug_init=debug_spot)
                            if print_log: print("is moving")
                            if imm == 0 or imm == 1: # (update) images equality or obj residual in prev_show 
                                a_current.append(bound)
                                b.append(np.uint8(np.logical_not(moving_next_zone)))
                                recover_current_bound.append(bound)
                            else: # (ignore it) probably the bound wasn't recognized properly
                                area_to_ignore = np.uint8(np.logical_or(prev_bound,bound))
                                a_current.append(area_to_ignore)
                                recover_current_bound.append(area_to_ignore)
                        elif np.any(static_prev_zone):
                            imm = spot_the_diff(static_prev_zone*prev_show, static_prev_zone*next_frame, log=print_log, show_work=show_work, debug_init=debug_spot)
                            if print_log: print("is getting smaller")
                            if imm == 0 or imm == 1: # (update) 
                                if print_log: print("and there is good stuff")
                                a_current.append(bound)
                                recover_current_bound.append(bound)
                            else:
                                a_current.append(prev_bound) # (ignore it)
                                recover_current_bound.append(prev_bound)
                        else:
                            if print_log: print("is getting bigger")
                            a_current.append(bound) # (ignore it)
                            recover_current_bound.append(bound)
            if len(a_current)>0:
                a_all = np.all(a_current, axis=0)
                a.append(a_all)
                if len(b_current)>0:
                    b_all = np.all(b_current, axis=0)
                    b.append(np.logical_and(np.logical_not(a_all), b_all))
            if len(recover_current_bound)>0:
                rec_all = np.all(recover_current_bound, axis=0) 
                contours, hierarchy = cv2.findContours(np.uint8(rec_all),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
                if len(contours)>1:
                    for idx in range(len(contours)):
                        canvas = np.zeros_like(next_frame, np.uint8)
                        cv2.drawContours(canvas, contours, idx, 1, -1)
                        recovery_bounds.append(canvas)
                else:
                    recovery_bounds.append(rec_all)
            
    if show_result:
        plt.figure(figsize=(20,10))
        plt.subplot(131); plt.title("Static area estim")
        plt.imshow(cv2.cvtColor(np.uint8(static_area*next_frame), cv2.COLOR_GRAY2RGB))
        plt.subplot(132); plt.title("Movement area estim")
        plt.imshow(cv2.cvtColor(np.uint8(movement_area*prev_frame), cv2.COLOR_GRAY2RGB))

        # apply good static pixels
    for old_zone in b:
        prev_show[old_zone>0]=(old_zone*prev_show)[old_zone>0]
    for non_static_zone in a:
        static_area[non_static_zone>0] = 0
    if len(a)>0: prev_show[static_area>0]=(static_area*next_frame)[static_area>0]

    if show_result:
        plt.subplot(132); plt.title("Bounds")
        plt.imshow(cv2.cvtColor(np.uint8(bounded_image), cv2.COLOR_GRAY2RGB))
        plt.title("Show")
        plt.subplot(133); plt.imshow(cv2.cvtColor(np.uint8(prev_show), cv2.COLOR_GRAY2RGB))
        plt.show()
        
    mean_prev.append(prev_show.copy())
    if len(mean_prev)>mean_prev_size:
        mean_prev.pop(0)

    output_video.append(np.hstack([frame.copy(),np.mean(mean_prev, axis=0).copy() if mean_prev_size>0 else prev_show.copy()]))
    
    #log
    print(colored("bounds: {0} - prev_bounds: {1} - recovery_bounds: {2}".format(len(bounds),len(prev_bounds),len(recovery_bounds)),"yellow"))
    print(colored("time: {}".format(time.time()-start),"yellow"))
    print()
    
    #restart
    prev_bounds = recovery_bounds if len(prev_bounds) > 0 else bounds # ERR
    bar.update((index-1)+1)

cap.release()
bar.finish()
out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, (frame.shape[1]*2,frame.shape[0]))

for i in range(len(output_video)):
    out.write(cv2.cvtColor(np.uint8(output_video[i]), cv2.COLOR_GRAY2RGB))
    out.write(cv2.cvtColor(np.uint8(output_video[i]), cv2.COLOR_GRAY2RGB))
out.release()